<a href="https://colab.research.google.com/github/Anuubhavvvv/Project-Python-machine-learning/blob/main/chatbot_projectfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install openai
!pip install gradio
!pip install sentence_transformers
!pip install huggingface_hub
!pip install json
!pip install os
!pip install numpy


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [23]:

import openai
import gradio as gr
import numpy as np
from sentence_transformers import SentenceTransformer, util
import json
import os
import time
import threading
import sqlite3
import pandas as pd


openai.api_key = ""

model = SentenceTransformer('all-MiniLM-L6-v2')

def init_db():
    conn = sqlite3.connect('knowledge_base.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS knowledge_base (
            id INTEGER PRIMARY KEY,
            question TEXT UNIQUE,
            answer TEXT
        )
    ''')
    conn.commit()
    conn.close()

init_db()


knowledge_base_amity ={ "hello": "Hello! How can I assist you today?",    "hi": "Hi there! What would you like to know?",
    "hey": "Hey! How can I help you?",
    "good morning": "Good morning! How can I assist you?",
    "good afternoon": "Good afternoon! What can I do for you?",
    "good evening": "Good evening! How may I help you?",
    "greetings": "Greetings! How can I assist you today?",
    "howdy": "Howdy! What can I do for you?",
    "what's up": "Not much! How can I assist you?",
    "yo": "Yo! What would you like to know?",
    "hi there": "Hi there! How can I help you?",
    "hello there": "Hello there! How can I assist you?",
    "good day": "Good day! How may I help you?",
    "morning": "Good morning! How can I assist you?",
    "afternoon": "Good afternoon! What can I do for you?",
    "evening": "Good evening! How can I assist you?",
    "what's good": "Everything’s good! How can I help you?",
    "how’s it going": "It's going well! How can I assist you?",
    "sup": "Not much! How can I help you?",
    "yo yo": "Yo! What would you like to know?",
    "hiya": "Hiya! How can I assist you today?",
    "how are you": "I'm good! How can I help you?",
    "how is everything": "Everything’s great! How can I assist you?",
    "how’s life": "Life’s good! How can I help you today?",
    "good to see you": "Good to see you too! How can I help?",
    "long time no see": "Indeed! How can I assist you today?",
    "what's new": "Not much! How can I assist you?",
    "hey buddy": "Hey there! How can I help you?",
    "hello friend": "Hello! What would you like to know?",
    "salutations": "Salutations! How can I assist you?",
    "yo what's up": "Yo! Not much, how can I help?",
    "hello everyone": "Hello! How can I assist you?",
    "hey there": "Hey! What would you like to know?",
    "g'day": "G'day! How can I help you today?",
    "greetings and salutations": "Greetings! How can I help you?",
    "what’s happening": "Not much! How can I assist you?",
    "how are things": "Things are good! How can I help you?",
    "hey mate": "Hey mate! How can I assist you?",
    "hello mate": "Hello! How can I assist you?",
    "hi friend": "Hi! How can I help you?",
    "what’s going on": "Not much! How can I assist you?",
    "hey hey": "Hey hey! How can I assist you today?",
    "yo yo yo": "Yo! What can I help you with?",
    "how’s your day": "My day’s going well! How can I assist you?",
    "hello": "Hello! How can I assist you today?",
    "hi": "Hi there! What would you like to know?",
    "hey": "Hey! How can I help you?",
    "good morning": "Good morning! How can I assist you?",
    "good afternoon": "Good afternoon! What can I do for you?",
    "good evening": "Good evening! How may I help you?",
    "greetings": "Greetings! How can I assist you today?",
    "what's up": "Not much! How can I help you today?",
    "howdy": "Howdy! What would you like to know?",
    "what can you do": "I can help you with information related to Amity University.",

    # Admissions and Enrollment
    "how do I apply to Amity University?": "You can apply online through the Amity University admissions portal.",
    "what are the eligibility criteria for undergraduate programs?": "Eligibility varies by program; please check the specific program details on the website.",
    "when is the application deadline?": "Application deadlines vary; please refer to the admissions section of the university's website.",
    "how do I check the status of my application?": "You can check your application status through the admissions portal using your login credentials.",
    "what documents do I need for admission?": "Required documents typically include your academic transcripts, identification proof, and application form.",
    "is there an entrance exam for admission?": "Some programs may require an entrance exam; please check the specific program requirements.",
    "how can I change my course preference after applying?": "You can contact the admissions office to request a change in your course preference.",
    "what is the process for international student admissions?": "International students must apply through the university's international admissions portal and provide additional documents.",
    "are there any scholarships for new students?": "Yes, there are several scholarships available based on merit and need. Check the scholarship section on the website.",

    # Course Information
    "what courses are offered this semester?": "You can view the list of courses on the Amity University course catalog page.",
    "how can I view the course syllabus?": "The syllabus for each course can be found on the course webpage in the online learning platform.",
    "what are the prerequisites for my chosen program?": "Prerequisites vary by program; check the academic catalog for specific details.",
    "how do I register for classes?": "Class registration is done through the Student Portal during the registration period.",
    "what is the process for dropping a class?": "To drop a class, log into the Student Portal and navigate to the 'Drop Classes' section.",
    "how do I get academic advising?": "Academic advising is available through the Academic Affairs office; you can schedule an appointment online.",
    "how can I access course materials?": "Course materials are uploaded by professors on the online learning platform.",
    "are there any online courses available?": "Yes, Amity University offers various online and hybrid courses. Check the course catalog for details.",
    "how do I declare my major?": "You can declare your major by filling out a form available at the Academic Affairs office.",

    # Academic Policies
    "what is the grading policy?": "The grading policy follows a standard A-F scale; please check the academic catalog for details.",
    "what happens if I fail a course?": "You will need to retake the course to meet program requirements, and it will affect your GPA.",
    "how do I appeal a grade?": "To appeal a grade, you must submit a formal appeal to the department chair within a specified timeframe.",
    "what is the attendance policy?": "Students are generally required to maintain at least 75% attendance in each course.",
    "how can I request a leave of absence?": "You can request a leave of absence by submitting a form to the Registrar's Office.",
    "what is the policy for academic integrity?": "The university has strict policies against academic dishonesty; violations may result in disciplinary action.",
    "are there guidelines for thesis or dissertation submissions?": "Yes, guidelines for thesis or dissertation submissions are provided by the Graduate School; please refer to their website.",
    "how is class participation graded?": "Class participation is typically graded based on attendance, engagement, and contributions to discussions.",

    # Financial Aid and Scholarships
    "how do I apply for financial aid?": "You can apply for financial aid through the financial aid office, providing necessary documentation.",
    "what scholarships are available for new students?": "Various scholarships are available; check the scholarship page for details and eligibility.",
    "how do I check my financial aid status?": "You can check your financial aid status on the Student Portal under the Financial Aid section.",
    "are there work-study opportunities available?": "Yes, Amity University offers work-study programs for eligible students. Check with the financial aid office for details.",
    "what is the process for accepting a scholarship offer?": "To accept a scholarship offer, you typically need to sign an acceptance form and submit it to the financial aid office.",
    "how do I request a tuition waiver?": "Tuition waivers can be requested through the Registrar's Office with proper documentation.",
    "what are the deadlines for scholarship applications?": "Scholarship application deadlines vary; please refer to the scholarship page for specific dates.",
    "can I apply for multiple scholarships?": "Yes, you can apply for multiple scholarships as long as you meet the eligibility criteria.",

    # Tuition and Fees
    "what is the tuition fee for my program?": "Tuition fees vary by program; please check the university's website for the fee structure.",
    "are there any additional fees I should be aware of?": "Additional fees may include laboratory fees, library fees, and other administrative charges.",
    "how can I pay my tuition fees?": "Tuition fees can be paid online through the Student Portal or at the finance office.",
    "what are the options for payment plans?": "The university offers payment plans for tuition fees; check with the Finance Office for details.",
    "what happens if I miss the tuition payment deadline?": "Late payments may incur additional fees and holds on your account.",
    "is there a late fee for tuition payments?": "Yes, a late fee is typically charged if tuition is not paid by the deadline.",
    "how do I check my outstanding balance?": "You can check your outstanding balance on the Student Portal under the Financial section.",

    # Housing and Accommodation
    "how do I apply for on-campus housing?": "On-campus housing applications are available on the Student Portal during the housing application period.",
    "what are the housing options available?": "Housing options include single rooms, shared rooms, and apartments. Please check the housing section for details.",
    "can I choose my roommate?": "Roommate preferences can be indicated on the housing application form, but placement is not guaranteed.",
    "what is the cost of on-campus accommodation?": "Housing costs vary, but typically start at ₹30,000 per semester.",
    "how do I report maintenance issues in my dorm?": "Maintenance issues can be reported through the housing portal or at the housing office.",
    "what are the rules for living in the dorms?": "Dorm rules include quiet hours, guest policies, and cleanliness standards. Refer to the housing handbook for specifics.",
    "are there laundry facilities in the dorms?": "Yes, laundry facilities are available in all dormitory buildings.",
    "how do I get a parking permit?": "Parking permits can be obtained from the campus security office or via the Student Portal.",

    # Health and Wellness
    "where can I find mental health services?": "Mental health services are offered at the campus wellness center, located near the gym.",
    "how do I schedule a doctor's appointment on campus?": "You can schedule appointments at the campus health center through the online portal.",
    "what health insurance options are available?": "Health insurance options are provided by the university; check the health services section for details.",
    "are there fitness classes available?": "Yes, the campus gym offers various fitness classes throughout the semester.",
    "how do I access health services?": "Health services are available to all students and can be accessed by visiting the campus health center.",

    # Extracurricular Activities
    "how do I join a student club?": "You can join clubs by attending the club fairs held at the beginning of each semester.",
    "what activities are available on campus?": "Activities include sports, cultural events, workshops, and student clubs.",
    "is there a gym on campus?": "Yes, Amity University has a fully equipped gym located in the student center.",
    "what are the gym hours?": "The gym is open from 6:00 AM to 10:00 PM daily.",
    "how do I get involved in student government?": "You can join student government by running for positions during election periods or volunteering for events.",

    # Other Miscellaneous Queries
    "what is the refund policy?": "Refunds are available if you withdraw before the end of the third week of classes. Please refer to the refund policy for details.",
    "how do I contact the admissions office?": "You can contact the admissions office via email, phone, or visit their office in person.",
    "where is the campus located?": "Amity University is located in Noida, Uttar Pradesh, India.",
    "what are the university's working hours?": "The university operates from 9:00 AM to 5:00 PM, Monday to Friday.",
    "is there a campus library?": "Yes, the campus has a central library with a wide range of resources for students.",
    "what are the class timings?": "Classes typically start at 9:00 AM and end at 5:00 PM. Please check your specific course schedule for exact timings.",
    "how can I check my class schedule?": "You can check your class schedule on the Amity University Student Portal under the 'My Courses' section.",
    "what are the office hours for professors?": "Office hours vary by department, but generally, professors are available 9:00 AM to 6:00 PM on weekdays. Check the departmental notice board for specific timings.",
    "how do I register for classes?": "Class registration is done through the Amity University Student Portal during the registration period announced by the university.",
    "what is the grading policy?": "The grading policy typically follows a standard A-F scale, with additional provisions for internal assessments.",
    "how do I drop a class?": "To drop a class, log into the Student Portal and navigate to the 'Drop Classes' section. Make sure to do it before the deadline.",
    "what is the deadline to drop a class?": "The deadline to drop a class without penalty is usually the end of the second week of the semester.",
    "what happens if I fail a class?": "If you fail a class, you will need to retake it to meet your program requirements, and it will impact your GPA.",
    "can I take summer classes?": "Yes, Amity University offers summer classes. Registration details are announced in April.",
    "what courses are offered this semester?": "You can view all available courses for this semester on the Amity University course catalog page.",
    "how do I apply for a course waiver?": "Course waivers can be applied for through the Academic Affairs office. Make sure to provide valid reasons and documentation.",
    "where can I find course materials?": "Course materials are uploaded by professors on the Amity University online learning platform.",
    "how do I access the course syllabus?": "The syllabus for each course is available on the course webpage on the online learning platform.",
    "what is the attendance policy?": "Attendance policies vary by course, but generally, students must maintain at least 75% attendance.",
    "can I attend classes online?": "Yes, many courses offer online or hybrid options. Please check your course details for more information.",
    "how do I declare my major?": "You can declare your major by filling out a form available at the Academic Affairs office.",
    "what are the required courses for my major?": "Required courses for your major can be found on the Amity University academic catalog.",
    "what is a prerequisite?": "A prerequisite is a course that must be completed before enrolling in a more advanced course.",
    "how do I get a transcript?": "Transcripts can be requested through the Registrar's Office or the Student Portal.",
    "how much is the tuition fee?": "The tuition fee varies by program. Please check the Amity University website for specific fee structures.",
    "when is the tuition payment deadline?": "The tuition payment deadline is usually the last Friday of the second week of classes.",
    "can I pay my fees in installments?": "Yes, Amity University allows payment plans for tuition fees. Check with the Finance Office for details.",
    "how do I apply for financial aid?": "You can apply for financial aid through the university's financial aid office, providing the necessary documentation.",
    "what scholarships are available?": "Amity University offers various scholarships based on merit and need. Details can be found on the university's website.",
    "what is the cost of housing?": "Housing costs vary, but on-campus accommodation typically starts at ₹30,000 per semester.",
    "how do I apply for a scholarship?": "Scholarship applications can be submitted online through the Amity University scholarship portal.",
    "what is the refund policy?": "Refunds are available if you withdraw before the end of the third week of classes. Please refer to the university's refund policy for details.",
    "what happens if I miss the payment deadline?": "Late payments may result in additional fees and holds on your account.",
    "are there any hidden fees?": "Additional fees may include laboratory fees, library fees, and other administrative charges depending on your courses.",
    "how do I check my outstanding balance?": "You can check your outstanding balance on the Student Portal under the 'Financial' section.",
    "what financial aid options are available?": "Financial aid options include scholarships, grants, and educational loans. Check with the financial aid office for specifics.",
    "how do I accept my financial aid award?": "You can accept your financial aid award through the Student Portal.",
    "can international students apply for financial aid?": "International students may have limited financial aid options, but some scholarships are available.",
    "what is the work-study program?": "The work-study program allows students to work part-time on campus while studying. Check with the financial aid office for details.",
    "how do I qualify for work-study?": "Eligibility for work-study is determined by your financial aid application.",
    "what happens if I don’t pay my tuition?": "Failure to pay tuition may lead to a hold on your account, preventing future registration.",
    "can I get a tuition discount?": "Tuition discounts may be available for specific groups such as athletes or top-performing students.",
    "how do I apply for a tuition waiver?": "Tuition waivers can be requested through the Registrar's Office with proper documentation.",
    "is there a late fee for tuition payments?": "Yes, there is typically a late fee of ₹1,000 for missing the tuition payment deadline.",
    "how do I join a student club?": "You can join student clubs by attending the club fairs held at the beginning of each semester.",
    "where is the library?": "The main library is located in the central campus building, and there are additional libraries in various departments.",
    "what time does the library close?": "The library generally closes at 10:00 PM on weekdays and 8:00 PM on weekends.",
    "how do I reserve a study room in the library?": "Study rooms can be reserved online through the library's booking system.",
    "what events are happening on campus this week?": "Check the events calendar on the Student Portal for a list of upcoming events and activities.",
    "is there a gym on campus?": "Yes, Amity University has a fully equipped gym located in the student center.",
    "what are the gym hours?": "The gym is open from 6:00 AM to 10:00 PM daily.",
    "how do I apply for on-campus housing?": "On-campus housing applications are available on the Student Portal during the housing application period.",
    "can I change my roommate?": "Requests to change roommates can be made through the housing office, subject to availability.",
    "what is the meal plan cost?": "The meal plan costs approximately ₹20,000 per semester and includes three meals daily.",
    "where is the cafeteria?": "The cafeteria is located in each block and offers various meal options.",
    "are there vegetarian options in the cafeteria?": "Yes, the cafeteria provides a variety of vegetarian and vegan meal options.",
    "how do I report a maintenance issue in my dorm?": "Maintenance issues can be reported through the housing portal or directly at the housing office.",
    "what activities are available on campus?": "Activities include sports, cultural events, workshops, and various student clubs.",
    "is there Wi-Fi on campus?": "Yes, free Wi-Fi is available across the campus, accessible with your student credentials.",
    "where can I park on campus?": "Parking is available in designated areas. Students need to apply for a parking permit.",
    "how do I get a parking permit?": "Parking permits can be obtained from the campus security office or via the Student Portal.",
    "are there bike racks on campus?": "Yes, bike racks are available near most academic buildings and dormitories.",
    "where can I find mental health services?": "Mental health services are offered at the campus wellness center, located near the gym.",
    "how do I get involved in student government?": "You can join student government by running for positions during election periods or volunteering for events.",
    "where is the registrar's office?": "The Registrar's Office is located in the main administrative building.",
    "how do I contact the admissions office?": "You can contact the admissions office via email or phone, or visit their office in person.",
    "how do I change my major?": "You can change your major by submitting a major change form through the academic advising office.",
    "how do I get a student ID?": "Student IDs can be obtained from the campus security office after completing your registration.",
    "what is the process for withdrawing from the university?": "To withdraw, you need to submit a withdrawal form through the Registrar’s Office and meet with an academic advisor.",
    "what are the library hours?": "The library is open from 8:00 AM to 10:00 PM on weekdays and 9:00 AM to 8:00 PM on weekends.",
    "how do I access the university's online resources?": "You can access online resources through the university's library portal",
"what is the placement process at Amity University?": "The placement process includes registration, pre-placement talks, and interviews. Students must be registered on the placement portal.",
    "how can I register for placements?": "You can register for placements through the Amity University placement portal, accessible via the Student Portal.",
    "what companies visit Amity University for placements?": "Various companies from diverse sectors visit Amity University for placements, including IT, finance, marketing, and consulting firms.",
    "what is the average package offered during placements?": "The average package varies by program, but it generally ranges from ₹4,00,000 to ₹10,00,000 per annum.",
    "how does Amity University support students in placements?": "Amity University offers support through workshops, resume building sessions, mock interviews, and industry interaction sessions.",
    "what is the eligibility criteria for placements?": "Eligibility criteria may include minimum academic performance, attendance, and completion of specific courses or internships.",
    "when do placements typically start?": "Placements usually start in the final year of the program, with companies visiting the campus from September to April.",
    "how do I prepare for placement interviews?": "Preparation involves researching companies, practicing common interview questions, and participating in mock interviews organized by the placement cell.",
    "are internships mandatory for placement eligibility?": "Internships may be mandatory for certain programs. Check with your program guidelines for specific requirements.",
    "can I participate in placements if I have backlogs?": "Students with backlogs may face restrictions in participating in placements, depending on the university's policies.",
    "how can I improve my chances of getting placed?": "Improving your skills, maintaining good grades, participating in extracurricular activities, and networking can enhance your placement prospects.",
    "does Amity University have a dedicated placement cell?": "Yes, Amity University has a dedicated placement cell that assists students with job placements and career guidance.",
    "what is the process for getting a job offer?": "Once you are shortlisted, the company will conduct interviews, and if selected, you will receive a formal job offer through the placement cell.",
    "what resources are available for career development?": "Amity University provides resources such as career counseling, workshops, and access to job listings on the placement portal.",
    "can I get help with my resume?": "Yes, the placement cell offers resume writing workshops and one-on-one sessions to help students create effective resumes.",
    "what are the most common roles offered during placements?": "Common roles include software developer, business analyst, marketing executive, and financial analyst.",
    "are there on-campus recruitment drives?": "Yes, Amity University conducts on-campus recruitment drives where multiple companies visit for interviews.",
    "how can I network with industry professionals?": "Students can network through alumni events, guest lectures, and workshops organized by the placement cell.",
    "what is the difference between placement and internship?": "Placement refers to full-time job offers after graduation, while internships are temporary work experiences, usually during studies.",
    "what should I do if I don’t get placed?": "If you don't get placed, consider seeking internships, improving your skills, or exploring alternative career paths through counseling services.",
    "are there opportunities for higher studies after completing my degree?": "Yes, many students pursue higher studies, including master's degrees or professional certifications after graduation.",
    "how can I access placement statistics for my program?": "Placement statistics can typically be found on the Amity University website or by contacting the placement cell directly.",
    "what skills are most valued by employers?": "Employers value communication skills, teamwork, problem-solving abilities, and technical skills relevant to the job role.",
    "are there workshops for interview preparation?": "Yes, the placement cell organizes workshops specifically designed for interview preparation.",
    "how do I find out about job openings?": "Job openings are posted on the placement portal, and students are also encouraged to network and reach out to alumni.",
    "can I apply for jobs off-campus?": "Yes, students can apply for off-campus jobs, but they should also keep the placement cell informed.",
    "what is the role of the alumni network in placements?": "The alumni network provides guidance, mentorship, and potential job opportunities through their connections in various industries.",
    "is there a fee for placement services?": "No, placement services provided by Amity University are free for students enrolled in the university.",
    "how often do placement cells communicate with students?": "The placement cell regularly communicates with students through emails, notices, and meetings to keep them updated.",
    "what happens during pre-placement talks?": "Pre-placement talks are sessions where companies present their organization, culture, and job opportunities to students.",
    "how can I get feedback after interviews?": "Students can request feedback from the placement cell or directly from the interviewing company after the process.",
    "do I need to dress formally for placement interviews?": "Yes, formal attire is typically recommended for placement interviews to make a professional impression.",
    "what resources are available for skill development?": "Resources include workshops, online courses, and access to learning platforms provided by the university.",
    "can I take a leave during placement season?": "Leave policies during placement season may vary. It's advisable to check with your department and placement cell.",
    "what if I want to change my career path after graduation?": "You can explore different career paths through further education, internships, or by seeking advice from career counselors.",
    "how can I get involved in placement cell activities?": "Students can volunteer for events or join committees organized by the placement cell to gain experience.",
    "what are the top companies that hire Amity graduates?": "Top companies include tech giants, financial institutions, and multinational corporations, depending on the specific program.",
    "what kind of support can I expect from faculty during placements?": "Faculty members often provide guidance, mentorship, and recommendation letters during the placement process.",
    "is there a mentorship program for students?": "Yes, Amity University often has mentorship programs connecting students with alumni and industry professionals.",
    "how do I prepare for group discussions during interviews?": "Preparation includes practicing common topics, understanding group dynamics, and being articulate in expressing your views.",
    "can I get placement assistance after graduation?": "Placement assistance is typically available for a limited time after graduation, but many services may still be accessible.",
    "are there any workshops on soft skills?": "Yes, the placement cell regularly organizes workshops focused on developing soft skills like communication and teamwork.",
    "what is the role of the career counseling cell?": "The career counseling cell provides guidance, resources, and support to help students make informed career decisions.",
    "how do I evaluate job offers?": "Consider factors such as salary, company culture, growth opportunities, and work-life balance when evaluating job offers.",
    "are there opportunities for international placements?": "Yes, some programs may offer international placements or internships as part of their curriculum.",
    "how can I stay updated on placement opportunities?": "Stay connected with the placement cell through emails, notifications, and by regularly checking the placement portal.",
    "what is the significance of the placement report?": "The placement report provides data on the number of students placed, average packages, and the companies involved, showcasing the university's placement success.",
    "can I defer my placement?": "Deferring placement opportunities may depend on university policies and specific circumstances, so check with the placement cell.",
    "what is the impact of internships on placements?": "Internships can enhance your resume, provide practical experience, and improve your chances of securing a job during placements.",
    "how important is networking for my career?": "Networking is crucial as it can lead to job opportunities, mentorship, and professional growth.",
    "what types of companies hire graduates from Amity University?": "Graduates are hired by a diverse range of companies, including startups, established firms, and multinational corporations.",
    "are there guest lectures by industry experts?": "Yes, guest lectures are often organized by the placement cell to expose students to industry insights and trends.",
    "can I access past placement records?": "Yes, past placement records may be available through the placement cell or on the university's website.",
    "how do I handle multiple job offers?": "Evaluate each offer carefully and consider your career goals, preferences, and values before making a decision.",
    "what is the significance of a good LinkedIn profile?": "A good LinkedIn profile can enhance your visibility to recruiters and help you connect with professionals in your field.",
    "what if I want to pursue entrepreneurship after graduation?": "Amity University provides resources and support for students interested in entrepreneurship through various initiatives.",
    "can I request a mentor from the placement cell?": "Yes, students can often request mentorship from the placement cell to gain insights and guidance.",
    "how can I develop my personal brand?": "Develop your personal brand by showcasing your skills, experiences, and values consistently across platforms.",
    "what is the process for getting a certificate of placement?": "Certificates are usually provided upon successful placement and can be requested through the placement cell.",
    "are there mock interviews conducted by the placement cell?": "Yes, mock interviews are conducted to help students practice and prepare for real interview scenarios.",
    "what are the common challenges faced during placements?": "Common challenges include competition, lack of experience, and uncertainty in career choices.",
    "how do I maintain my mental well-being during the placement process?": "Prioritize self-care, seek support from peers or counselors, and maintain a balanced perspective during the placement process.",
    "what types of fines can I incur at Amity University?": "Fines can be incurred for late submission of assignments, library fines for overdue books, and penalties for not adhering to university policies.",
    "what is the fine for late submission of assignments?": "The fine for late submission of assignments typically varies by department, but may range from ₹100 to ₹500 per day.",
    "are there fines for not attending lectures?": "Yes, fines may be imposed for low attendance, depending on the program's attendance policy.",
    "how do I know if I have incurred any fines?": "You can check for any outstanding fines through the student portal or by contacting the administration office.",
    "what is the penalty for academic dishonesty?": "Penalties for academic dishonesty can include failing grades, suspension, or expulsion, depending on the severity of the offense.",
    "how can I appeal a fine?": "To appeal a fine, you must submit a written appeal to the administration along with any supporting documentation within a specified timeframe.",
    "what happens if I don't pay my fines?": "Failure to pay fines may result in restrictions on accessing certain university services or participation in placements.",
    "are there fines for missing examinations?": "Yes, missing examinations without valid justification may incur fines or result in failing the course.",
    "can I get a refund for fines paid?": "Refunds for fines are generally not available, but you can inquire about specific circumstances with the administration.",
    "how are library fines calculated?": "Library fines are typically calculated based on the number of overdue days and the fine rate set by the library.",
    "what is the process for paying fines?": "Fines can usually be paid through the student portal or at the administration office.",
    "are there penalties for violating university conduct rules?": "Yes, violations of conduct rules can lead to penalties, including fines, warnings, or disciplinary action.",
    "what is the fine for not returning library books on time?": "The fine for overdue library books may vary, typically around ₹10 per day per book.",
    "can I contest a fine for a late assignment?": "You may contest a fine for a late assignment by providing a valid explanation to the faculty or administration.",
    "what are the consequences of not adhering to university policies?": "Consequences may include fines, suspension, or other disciplinary actions, depending on the policy violation.",
    "are there any exemptions for fines?": "Exemptions may be available in certain cases, such as medical emergencies or other justifiable reasons. Check with the administration.",
    "how often are fines reviewed or updated?": "Fines and penalties are reviewed periodically by the university administration to ensure they align with current policies.",
    "what documentation is needed to appeal a fine?": "Documentation may include a letter of explanation, supporting evidence, and any relevant forms provided by the university.",
    "can I be fined for damaging university property?": "Yes, fines may be imposed for damages to university property, with costs assessed based on the extent of the damage.",
    "are there fines for not following lab safety rules?": "Yes, failing to adhere to lab safety rules may result in fines or other disciplinary actions.",
    "what is the penalty for plagiarism?": "Plagiarism may result in penalties ranging from a failing grade on the assignment to suspension, depending on the severity.",
    "how can I find out about the fine policies?": "Fine policies can usually be found in the student handbook or on the university's official website.",
    "what should I do if I can't afford to pay a fine?": "If you are unable to pay a fine, discuss your situation with the administration to explore possible options for assistance.",
    "is there a grace period for paying fines?": "Some fines may have a grace period, but this varies by type of fine. Check with the administration for specifics.",
    "how does the university inform students about fines?": "Students are typically informed about fines via email notifications, the student portal, or official announcements.",
    "are there any fines for late payment of tuition fees?": "Yes, late tuition fee payments may incur fines, usually specified in the university's fee structure.",
    "what is the penalty for exceeding the allowed number of absences?": "Exceeding the allowed number of absences may result in fines, academic penalties, or loss of credit for the course.",
    "can fines be paid in installments?": "Payment options for fines, including installments, may be available but should be discussed with the administration office.",
    "what is the procedure for resolving disputes over fines?": "Disputes can be resolved by contacting the administration office and providing necessary documentation to support your case.",
    "are there any fines for not participating in mandatory events?": "Yes, fines may be imposed for non-participation in mandatory events or activities as outlined in your program requirements.",
    "how can I stay informed about changes in fines and penalties?": "Stay informed by regularly checking the student portal, university announcements, or the official website.",
    "is there a list of common fines and their amounts?": "A list of common fines and their amounts can usually be found in the student handbook or the university's official policies.",
    "what is the fine for using prohibited items in the library?": "Fines for using prohibited items in the library may vary, but could include a flat fee or removal of library privileges.",
    "can I get a fine for not completing my projects on time?": "Yes, fines may be imposed for not completing projects by their deadlines, as per the department's guidelines.",
    "what happens if I fail to comply with a penalty?": "Failure to comply with a penalty may result in further disciplinary action or restrictions on university services.",
    "are there fines for disruptive behavior in class?": "Yes, disruptive behavior in class can lead to fines or other disciplinary actions as per the university's code of conduct.",
    "what is the process for reporting fines to the administration?": "Fines are usually reported through the student portal or directly to the administration office for processing.",
    "how do I get more information about specific fines?": "For specific fines, contact the administration office or check the university's official guidelines.",
    "can I request a waiver for a fine?": "Waivers for fines may be considered under certain circumstances. You should submit a formal request to the administration.",
    "are there fines for using mobile phones in classrooms?": "Yes, there may be fines for using mobile phones during lectures, depending on the faculty's policy.",
    "what is the consequence of late registration for classes?": "Late registration may incur a fine and could affect your course schedule and availability.",
    "are there fines for violating campus safety rules?": "Yes, violations of campus safety rules can lead to fines or disciplinary actions as determined by the university.",
    "what is the fine for not adhering to dress code policies?": "Fines for not adhering to dress code policies may vary by department, typically ranging from ₹100 to ₹500.",
    "how can I avoid incurring fines?": "To avoid fines, ensure timely submission of assignments, adhere to university policies, and maintain good attendance.",
    "is there a record of fines incurred available for students?": "Yes, students can access their records of incurred fines through the student portal.",
    "can fines affect my academic standing?": "Yes, unpaid fines may affect your academic standing and eligibility for certain programs or placements.",
    "are there any scholarships available for students facing financial difficulties due to fines?": "Some scholarships may be available for students facing financial difficulties. Check with the financial aid office for options.",
    "what is the code of conduct for students?": "The Code of Conduct outlines the expected behaviors and responsibilities of students, including maintaining integrity, respect, and accountability in all academic and non-academic activities.",
    "what are the consequences of violating the code of conduct?": "Violating the Code of Conduct can result in disciplinary actions, which may include warnings, probation, suspension, or expulsion, depending on the severity of the violation.",
    "how can I report a violation of the code of conduct?": "Students can report violations to the Student Affairs office or through the university's online reporting system, providing all relevant details and evidence.",
    "what actions are considered academic misconduct?": "Academic misconduct includes cheating, plagiarism, fabrication of data, and unauthorized collaboration on assignments or exams.",
    "what are the penalties for plagiarism?": "Penalties for plagiarism can range from receiving a failing grade on the assignment to expulsion from the university, depending on the severity and intent.",
    "what is the process for a disciplinary hearing?": "The process involves an investigation, followed by a hearing where both the accused and the complainant can present their case. A decision will be made based on the evidence presented.",
    "how can I appeal a disciplinary decision?": "Students can appeal a disciplinary decision by submitting a written appeal to the designated university authority within a specified timeframe, usually within 10 days of receiving the decision.",
    "what types of behavior are prohibited on campus?": "Prohibited behaviors include harassment, discrimination, substance abuse, vandalism, and any actions that disrupt the educational environment.",
    "what is the policy on substance abuse?": "Amity University maintains a strict no-tolerance policy regarding drug and alcohol abuse. Violations may lead to severe disciplinary actions, including expulsion.",
    "what is the role of the Student Disciplinary Committee?": "The Student Disciplinary Committee is responsible for reviewing cases of misconduct, determining appropriate sanctions, and ensuring that due process is followed.",
    "can I be suspended for attending a party where underage drinking occurs?": "Yes, if you are found to be facilitating or participating in underage drinking, disciplinary actions may be taken, including suspension.",
    "what happens if I receive a warning?": "A warning serves as a formal notice of your misconduct. Continued violations may lead to more severe disciplinary actions.",
    "how does the university handle harassment claims?": "Harassment claims are taken seriously and investigated promptly. Disciplinary actions may be imposed if a claim is substantiated.",
    "what are the rules regarding student organizations?": "Student organizations must adhere to university policies, including maintaining a respectful environment, financial accountability, and ensuring activities are lawful.",
    "what is the procedure for handling disputes between students?": "Students are encouraged to resolve disputes informally, but if necessary, they can seek mediation from the Student Affairs office or initiate a formal complaint.",
    "are there specific dress code policies?": "Yes, students are expected to dress appropriately for a learning environment. Inappropriate attire may lead to a warning or request to change.",
    "what disciplinary actions can be taken for disruptive behavior in class?": "Disruptive behavior may result in a warning, removal from class, or more severe penalties depending on the frequency and nature of the disruption.",
    "is there a policy for digital conduct and social media?": "Yes, students are expected to uphold the university's values online. Inappropriate posts or behavior on social media can lead to disciplinary action.",
    "what is the policy on cheating during exams?": "Cheating is considered a serious violation. Consequences may include failing the exam, failing the course, or expulsion, depending on the case.",
    "how often are the disciplinary policies reviewed?": "Disciplinary policies are reviewed regularly to ensure they remain effective and relevant to the university's mission and values.",
    "can students be punished for off-campus behavior?": "Yes, students can face disciplinary action for off-campus behavior that adversely affects the university community or its reputation.",
    "what are the guidelines for student conduct during university events?": "Students are expected to behave respectfully and responsibly during university events, adhering to all rules and policies.",
    "are there any rehabilitation programs for students who violate policies?": "Yes, the university may offer rehabilitation or counseling programs for students who violate policies related to substance abuse or behavioral issues.",
    "who can I contact for more information about the code of conduct?": "For more information, students can contact the Student Affairs office or visit the university's official website for detailed policies and procedures.",
"what types of clubs are available at Amity University?": "Amity University offers a wide range of clubs, including cultural, technical, literary, music, dance, sports, and social service clubs. Students can join any club based on their interests.",
    "how do I join a club at Amity University?": "Students can join clubs by signing up during club fairs held at the beginning of each semester or by contacting club leaders directly through the university's student portal.",
    "what is the Amity Cultural Fest?": "The Amity Cultural Fest is an annual event that showcases the cultural diversity of the university. It includes various competitions, performances, workshops, and exhibitions.",
    "how often does the university hold sports events?": "Amity University organizes sports events throughout the year, including inter-university competitions, annual sports day, and various tournaments in different sports.",
    "what are the major academic events at Amity University?": "Major academic events include conferences, seminars, guest lectures, and workshops, focusing on various fields of study and inviting industry experts.",
    "when is the annual sports day held?": "The annual sports day is usually held in the spring semester, typically in March or April. Exact dates are announced in advance through the university's official channels.",
    "what cultural activities are organized throughout the year?": "Cultural activities include dance, music, drama, art exhibitions, and festivals celebrating various cultural events such as Holi, Diwali, and Eid.",
    "is there a student council at Amity University?": "Yes, Amity University has a student council that represents student interests, organizes events, and provides a platform for student voices within the university.",
    "what are the benefits of joining a club?": "Joining a club helps students develop leadership skills, network with peers, participate in events, and gain experience in their areas of interest.",
    "are there any leadership programs for students?": "Yes, the university offers various leadership development programs and workshops aimed at enhancing students' leadership skills and preparing them for future roles.",
    "what is the process for starting a new club?": "To start a new club, students must submit a proposal to the Student Affairs office, outlining the club's purpose, activities, and proposed membership.",
    "how are clubs funded at Amity University?": "Clubs receive funding from the university based on their activities and events. Clubs may also organize fundraising events to support their initiatives.",
    "what is the Amity Youth Festival?": "The Amity Youth Festival is a significant event that brings together students from various campuses, featuring competitions, cultural performances, and a platform for showcasing talent.",
    "are there any professional development events?": "Yes, the university organizes workshops, internships, and guest lectures aimed at enhancing students' professional skills and preparing them for the job market.",
    "how can I stay updated on events at Amity University?": "Students can stay updated on events by regularly checking the university's official website, student portal, and social media channels.",
    "is there a cultural exchange program?": "Yes, Amity University has cultural exchange programs that allow students to participate in international events, workshops, and collaborations with partner institutions.",
    "what role do student volunteers play in university events?": "Student volunteers assist in organizing and managing events, gaining valuable experience while contributing to the success of the university's activities.",
    "how does Amity celebrate its annual day?": "Amity celebrates its annual day with a grand event that includes cultural performances, awards ceremonies, and activities showcasing student talent.",
    "are there any environmental clubs at Amity University?": "Yes, there are environmental clubs focused on sustainability, awareness campaigns, and initiatives aimed at promoting environmental conservation among students.",
    "what types of sports teams are available?": "Amity University has various sports teams for cricket, football, basketball, volleyball, athletics, and more, competing in inter-university tournaments.",
    "is there a photography club?": "Yes, there is a photography club where students can explore their interest in photography, participate in workshops, and showcase their work in exhibitions.",
    "what events does the university organize for freshers?": "The university organizes orientation programs and fresher's welcome events to help new students acclimatize to campus life, meet peers, and learn about available resources.", "where can I find the library at Amity University?": "The library is located in the central campus building, featuring a vast collection of books, journals, and digital resources.",
    "what are the library hours?": "The library is open from 8:00 AM to 10:00 PM on weekdays and 9:00 AM to 8:00 PM on weekends.",
    "is there a computer lab available for students?": "Yes, there are multiple computer labs equipped with the latest software and hardware for student use across various campuses.",
    "how can I access online resources from the library?": "Students can access online resources through the university's library portal using their student credentials.",
    "are there study rooms available in the library?": "Yes, the library has study rooms that can be reserved online for group study sessions or quiet work.",
    "what sports facilities are available at Amity University?": "Amity University offers facilities for various sports, including cricket fields, basketball courts, tennis courts, swimming pools, and gymnasiums.",
    "is there a health center on campus?": "Yes, Amity University has a health center that provides medical services, including consultations, first aid, and emergency care.",
    "where can I find mental health services?": "Mental health services are available at the campus wellness center, which offers counseling and support for students.",
    "how do I apply for on-campus housing?": "On-campus housing applications are available on the Student Portal during the designated housing application period.",
    "what amenities are included in the hostel?": "Hostel amenities typically include furnished rooms, Wi-Fi access, common areas, laundry facilities, and meal services.",
    "is there a cafeteria on campus?": "Yes, there are cafeterias and food courts offering a variety of meal options, including vegetarian and non-vegetarian dishes.",
    "are there recreational facilities available?": "Yes, the university offers recreational facilities, including game rooms, outdoor sports areas, and student lounges.",
    "how can I access printing and photocopying services?": "Printing and photocopying services are available in designated areas on campus, usually near the library or computer labs.",
    "is there Wi-Fi available throughout the campus?": "Yes, free Wi-Fi is accessible across the campus for students and faculty using their university credentials.",
    "what transportation options are available for students?": "Amity University provides shuttle services between campuses and nearby areas for the convenience of students.",
    "how can I book a study room in the library?": "Study rooms can be reserved online through the library's booking system available on the university's website.",
    "are there any quiet study areas on campus?": "Yes, quiet study areas are available in the library and designated spots across campus for focused study.",
    "what is the student services office responsible for?": "The Student Services office assists with academic advising, career services, housing support, and student life activities.",
    "where can I find information about scholarships and financial aid?": "Information regarding scholarships and financial aid is available on the university's financial aid office webpage.",
    "is there an IT helpdesk for technical issues?": "Yes, there is an IT helpdesk on campus where students can seek assistance with technical issues related to university services.",
    "what types of workshops are available for skill development?": "The university offers various workshops focused on professional skills, leadership, communication, and technical skills throughout the year.",
    "are there any photography or art studios available?": "Yes, there are dedicated art and photography studios for students enrolled in related programs to use for their projects.",
    "how do I report a maintenance issue in the dormitory?": "Maintenance issues can be reported through the housing portal or by contacting the housing office directly.",
    "what resources are available for international students?": "International students have access to specific resources, including orientation programs, visa assistance, and cultural exchange initiatives.",
 "who founded Amity University?": "Amity University was founded by Dr. Ashok K. Chauhan in 2005.",
    "where is the main campus of Amity University located?": "The main campus of Amity University is located in Noida, Uttar Pradesh, India.",
    "how many campuses does Amity University have?": "Amity University has multiple campuses across India, including in Noida, Greater Noida, Lucknow, Jaipur, and Gwalior.",
    "what are the main branches of study at Amity University?": "Amity University offers a wide range of programs across various branches, including Arts, Science, Business, Law, Engineering, and more.",
    "is Amity University recognized by the UGC?": "Yes, Amity University is recognized by the University Grants Commission (UGC) of India.",
    "what types of degrees does Amity University offer?": "Amity University offers undergraduate, postgraduate, and doctoral degrees in various fields.",
    "does Amity University have international collaborations?": "Yes, Amity University has several international collaborations with universities and institutions around the world for academic exchange and research.",
    "what is the vision of Amity University?": "The vision of Amity University is to provide high-quality education that contributes to the intellectual, cultural, and social advancement of society.",
    "when was Amity University established?": "Amity University was established in the year 2005.",
    "what is the motto of Amity University?": "The motto of Amity University is 'Where the Future Begins'.",
    "does Amity University offer distance learning programs?": "Yes, Amity University offers distance learning programs through Amity University Online.",
    "what are the admission criteria for undergraduate programs?": "Admission criteria for undergraduate programs typically include qualifying examinations such as the Amity JEE, AUEEE, or other relevant entrance tests.",
    "is there an entrance exam for postgraduate programs at Amity University?": "Yes, many postgraduate programs require candidates to qualify for entrance exams, which may include the GMAT, GRE, or university-specific tests.",
    "how many students are enrolled at Amity University?": "Amity University has a diverse student population, with over 100,000 students enrolled across its various campuses.",
    "does Amity University have a placement cell?": "Yes, Amity University has a dedicated placement cell that assists students with internships and job placements.",
    "what extracurricular activities are offered at Amity University?": "Amity University offers various extracurricular activities, including sports, cultural events, clubs, and student organizations.",
    "what is the campus size of Amity University Noida?": "The Amity University Noida campus is spread over approximately 100 acres, equipped with modern facilities.",
    "does Amity University provide accommodation for students?": "Yes, Amity University provides on-campus accommodation options for students, including hostels with various amenities.",
    "how can I contact the admissions office at Amity University?": "The admissions office can be contacted via email, phone, or in-person visit at the university's main campus.",
    "does Amity University have a student exchange program?": "Yes, Amity University has student exchange programs with several international universities for enhancing global exposure.",
    "what is the ranking of Amity University?": "Amity University has consistently ranked among the top private universities in India by various national and international ranking agencies.",
    "does Amity University have a research facility?": "Yes, Amity University has established research facilities and centers of excellence for various fields of study.",
"who are some prominent alumni of Amity University?": "Some prominent alumni of Amity University include: 1. Dr. A. P. J. Abdul Kalam - Former President of India, 2. Aishwarya Rai Bachchan - Actress and Miss World, 3. Anushka Sharma - Bollywood Actress and Producer, 4. Sandeep Maheshwari - Entrepreneur and Motivational Speaker, 5. Arun Jaitley - Former Finance Minister of India, 6. Smriti Irani - Union Minister and Politician."
 ,"Who is the Director of the AIIT program?": "Dr. Rekha is the Director, overseeing the academic and administrative functions of the program.",
    "Who are the Program Leaders for the MCA at AIIT?": [
        "Dr. Monika Sharma - Program Leader, responsible for academic coordination.",
        "Dr. Prashant Srivastava - Program Leader, coordinating curriculum and academic initiatives.",
        "Dr. Sudhriti Sen Gupta - Program Leader, leading academic programs and faculty support."
    ],
    "Who is the Deputy Director of the AIIT program?": "Dr. Sapna Sinha serves as the Deputy Director, assisting with program management.",
    "Who is in charge of examinations for AIIT?": "Dr. Himanshu Gupta is the Examination In-Charge, overseeing the examination processes for students.",
    "Who is the Placement Coordinator for the AIIT program?": "Dr. Neetu Mittal is the Placement Coordinator, facilitating placements and career opportunities for students.",
    "Who is the Dean of Student Affairs?": "Dr. Laxmi Ahuja is the Dean of Student Affairs, managing student services and welfare.",
 "Who is the Founder of Amity University?": "Dr. Ashok K. Chauhan is the Founder of Amity University, known for his contributions to education and industry.",
    "Who is the Chancellor of Amity University?": "Dr. (Mrs.) Amita Chauhan serves as the Chancellor, overseeing the academic and administrative functions of the university.",
    "Who is the Vice-Chancellor of Amity University?": "Prof. (Dr.) Balvinder Shukla is the Vice-Chancellor, responsible for the overall academic and operational management.",
    "Who is the Pro Vice-Chancellor of Amity University?": "Dr. (Mrs.) Jaya Chandiramani is the Pro Vice-Chancellor, assisting the Vice-Chancellor in academic matters.",
    "Who is the Registrar of Amity University?": "Dr. (Mr.) Surender Singh is the Registrar, managing academic records and student services.",
    "Who is the Dean of Student Affairs?": "Dr. Laxmi Ahuja is the Dean of Student Affairs, responsible for student welfare and activities.",
    "Who is the Director of Admissions?": "Dr. Neeraj Sood is the Director of Admissions, overseeing the admissions process and student recruitment.",
    "Who is the Director of Finance?": "Mr. Sandeep Kumar is the Director of Finance, managing the financial operations and budgeting.",
    "Who is the Director of Human Resources?": "Dr. Kavita Raghunath is the Director of Human Resources, responsible for recruitment and employee relations.",
    "Who is the Director of Research and Innovation?": "Dr. Rakesh Bhatnagar is the Director of Research and Innovation, leading research initiatives and projects.",
  "What special practices does Amity University implement for student engagement?":
        "Amity University promotes various student engagement practices, including hands-on learning experiences, interactive workshops, and industry collaborations. The university also encourages participation in extracurricular activities and provides platforms for students to showcase their talents.",

    "How does Amity University facilitate experiential learning?":
        "Amity University emphasizes experiential learning through internships, live projects, and case studies. Students are encouraged to apply theoretical knowledge in practical scenarios, enhancing their understanding and employability.",

    "What are the unique teaching methodologies used at Amity?":
        "Amity employs a mix of traditional and modern teaching methodologies, including flipped classrooms, project-based learning, and the use of technology in education. Faculty members incorporate real-world problems into the curriculum to prepare students for industry challenges.",

    "Are there any special programs for research and innovation at Amity?":
        "Amity University fosters research and innovation through dedicated centers and institutes that focus on various fields. Students and faculty are encouraged to participate in research projects, seminars, and conferences to promote a culture of inquiry.",

    "How does Amity University support entrepreneurship among students?":
        "The university has an Entrepreneurship Development Cell (EDC) that provides support, mentorship, and resources for students interested in starting their own ventures. Workshops, incubation centers, and networking events are organized to help aspiring entrepreneurs.",

    "What initiatives does Amity University take for social responsibility?":
        "Amity engages in various social responsibility initiatives, including community service projects, environmental sustainability programs, and partnerships with NGOs. Students are encouraged to participate in these initiatives to develop a sense of social responsibility.",

    "What kind of skill development programs are offered at Amity?":
        "Amity offers numerous skill development programs, including workshops on soft skills, leadership, communication, and technical skills. These programs are designed to enhance employability and prepare students for the competitive job market.",
    "What major cultural events are organized at Amity University?":
        "Amity University hosts several cultural events throughout the year, including 'Amity Fest', which is a grand annual cultural festival featuring music, dance, art exhibitions, and food stalls. Other notable events include 'Aagaz', a cultural fest focusing on performing arts, and 'Amity Talent Search', showcasing students' artistic talents across various disciplines.",

    "Are there any annual fests at Amity University?":
        "Yes, Amity University hosts annual fests like 'Amity Youth Festival' and 'Aagaz', where students participate in various cultural activities, competitions, and performances, allowing them to showcase their talents and engage with peers from different disciplines.",

    "How does Amity University celebrate festivals?":
        "The university celebrates various cultural and national festivals, including Diwali, Holi, Christmas, and Eid, with dedicated events that include performances, food stalls, and traditional activities. These celebrations promote cultural diversity and foster unity among students.",

    "What types of performances are included in cultural events?":
        "Cultural events at Amity include a wide range of performances such as dance, drama, music concerts, and fashion shows. Students can participate or watch these performances, which are aimed at enhancing the cultural fabric of the university.",

    "Are there any workshops or competitions related to arts at Amity?":
        "Yes, Amity organizes various workshops and competitions related to arts, music, and dance during its cultural events. These activities encourage students to develop their skills and gain practical experience in their areas of interest.",   "who is the director at Amity University?": "Dr. Rekha is the director.",
    "can you tell me who is the director in charge?": "The director in charge is Dr. Rekha.",
    "who is the current director for the AIIT program?": "Dr. Rekha serves as the current director for the AIIT program.",
    "who serves as the deputy director at Amity University?": "Dr. Sapna Sinha is the deputy director.",
    "who is the deputy director of the AIIT program?": "Dr. Sapna Sinha is the deputy director of the AIIT program.",

    "who are the program leaders for the MCA course?": "Dr. Prashant Srivastava, Dr. Monika Sharma, and Dr. Sudhriti Sen Gupta are the program leaders for the MCA course.",
    "can you provide names of the MCA program leaders at Amity University?": "The MCA program leaders are Dr. Prashant Srivastava, Dr. Monika Sharma, and Dr. Sudhriti Sen Gupta.",
    "which faculty members lead the MCA program?": "Dr. Prashant Srivastava, Dr. Monika Sharma, and Dr. Sudhriti Sen Gupta are the faculty leading the MCA program.",
    "who is responsible for overseeing the MCA program?": "The MCA program is overseen by Dr. Prashant Srivastava, Dr. Monika Sharma, and Dr. Sudhriti Sen Gupta.",

    "who are the faculty members leading the BCA program?": "Dr. Komal Saxena, Dr. Alka Chaudhary, Dr. Rachna Jain, and Dr. Mayank Sharma are the program leaders for the BCA program.",
    "can you tell me the names of the program leaders for BCA?": "The program leaders for BCA are Dr. Komal Saxena, Dr. Alka Chaudhary, Dr. Rachna Jain, and Dr. Mayank Sharma.",
    "who is in charge of the BCA course at Amity University?": "The BCA course is managed by Dr. Komal Saxena, Dr. Alka Chaudhary, Dr. Rachna Jain, and Dr. Mayank Sharma.",

    "who leads the BSc IT program?": "Dr. Rashmi Vashisht and Dr. Gurpreet Kaur lead the BSc IT program.",
    "can you provide information about the program leaders for BSc IT?": "The BSc IT program leaders are Dr. Rashmi Vashisht and Dr. Gurpreet Kaur.",
    "who are the faculty members responsible for the BSc IT course?": "The faculty members responsible for the BSc IT course are Dr. Rashmi Vashisht and Dr. Gurpreet Kaur.",

    "who is the examination in-charge at Amity University?": "Dr. Himanshu Gupta is the examination in-charge.",
    "can you tell me the name of the person overseeing examinations?": "The person overseeing examinations is Dr. Himanshu Gupta.",
    "who manages the exam procedures at Amity University?": "Dr. Himanshu Gupta manages the exam procedures.",

    "who coordinates placements for the AIIT program?": "Neetu Mittal is the placement coordinator for the AIIT program.",
    "can you tell me who is responsible for placement activities at Amity University?": "The placement coordinator for the AIIT program is Neetu Mittal.",
    "who is the placement coordinator for the AIIT program?": "Neetu Mittal is the placement coordinator for the AIIT program.",

    "who is the dean of student affairs at Amity University?": "Laxmi Ahuja is the dean of student affairs.",
    "can you tell me who oversees student affairs?": "Laxmi Ahuja oversees student affairs.",
    "who is responsible for managing student-related issues at Amity University?": "The management of student-related issues is overseen by Laxmi Ahuja.",

    "can you give me a list of program leaders for various courses at Amity University?": "The program leaders include Dr. Rekha (Director), Dr. Sapna Sinha (Deputy Director), Dr. Prashant Srivastava, Dr. Monika Sharma, Dr. Sudhriti Sen Gupta (MCA), Dr. Komal Saxena, Dr. Alka Chaudhary, Dr. Rachna Jain, Dr. Mayank Sharma (BCA), Dr. Rashmi Vashisht, and Dr. Gurpreet Kaur (BSc IT).",
    "who are the academic leaders for different programs in the university?": "The academic leaders are Dr. Rekha, Dr. Sapna Sinha, Dr. Prashant Srivastava, Dr. Monika Sharma, Dr. Sudhriti Sen Gupta, Dr. Komal Saxena, Dr. Alka Chaudhary, Dr. Rachna Jain, Dr. Mayank Sharma, Dr. Rashmi Vashisht, and Dr. Gurpreet Kaur.",
    "which faculty members hold program leader positions at Amity University?": "The program leaders at Amity University include Dr. Prashant Srivastava, Dr. Monika Sharma, Dr. Sudhriti Sen Gupta, Dr. Komal Saxena, Dr. Alka Chaudhary, Dr. Rachna Jain, Dr. Mayank Sharma, Dr. Rashmi Vashisht, and Dr. Gurpreet Kaur.",

    "what positions do Dr. Rekha, Dr. Monika Sharma, Dr. Sapna Sinha, and others hold?": "Dr. Rekha is the Director, Dr. Sapna Sinha is the Deputy Director, and Dr. Monika Sharma is a program leader for MCA.",
    "can you provide details about the faculty in charge of various programs at Amity University?": "The faculty in charge include Dr. Rekha (Director), Dr. Sapna Sinha (Deputy Director), Dr. Prashant Srivastava, Dr. Monika Sharma, Dr. Sudhriti Sen Gupta (MCA), Dr. Komal Saxena, Dr. Alka Chaudhary, Dr. Rachna Jain, Dr. Mayank Sharma (BCA), Dr. Rashmi Vashisht, and Dr. Gurpreet Kaur (BSc IT).",
    "who are the key faculty members at Amity University and what roles do they play?": "Key faculty members include Dr. Rekha (Director), Dr. Sapna Sinha (Deputy Director), Dr. Prashant Srivastava, Dr. Monika Sharma, Dr. Sudhriti Sen Gupta (MCA), Dr. Komal Saxena, Dr. Alka Chaudhary, Dr. Rachna Jain, Dr. Mayank Sharma (BCA), Dr. Rashmi Vashisht, and Dr. Gurpreet Kaur (BSc IT)."
}

# File paths
interaction_log_path = 'interaction_log.txt'
feedback_storage_path = 'feedback_storage.json'

# Function to log interactions
def log_interaction(user_input, bot_response):
    with open(interaction_log_path, "a") as log_file:
        log_file.write(f"User: {user_input}\nBot: {bot_response}\n\n")

# Function to store user feedback
def provide_feedback(user_input, feedback):
    feedback_storage = {}
    if os.path.exists(feedback_storage_path):
        with open(feedback_storage_path, 'r') as file:
            feedback_storage = json.load(file)

    feedback_storage[user_input] = feedback
    with open(feedback_storage_path, 'w') as file:
        json.dump(feedback_storage, file)

# Function to update the knowledge base
def update_knowledge_base(question, answer):
    conn = sqlite3.connect('knowledge_base.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT OR REPLACE INTO knowledge_base (question, answer)
        VALUES (?, ?)
    ''', (question.lower(), answer))
    conn.commit()
    conn.close()

# Function to find the most similar question in the knowledge base
def find_most_similar_question(user_input, threshold=0.7):
    user_input_embedding = model.encode(user_input.lower())
    knowledge_base_questions = list(knowledge_base_amity.keys())
    knowledge_base_embeddings = model.encode(knowledge_base_questions)

    similarities = util.pytorch_cos_sim(user_input_embedding, knowledge_base_embeddings)
    most_similar_index = np.argmax(similarities)
    similarity_score = similarities[0][most_similar_index].item()

    if similarity_score >= threshold:
        return knowledge_base_questions[most_similar_index]
    else:
        return None

# Function to generate responses based on user input
def generate_responses(user_input, num_responses=2):
    most_similar_question = find_most_similar_question(user_input)

    if most_similar_question is None:
        return ["I'm sorry, I couldn't find an answer. Can you please rephrase your question?"]

    response = knowledge_base_amity[most_similar_question]
    return [response] * num_responses  # Repeat the response for demo purposes

# Function to handle responses and feedback
def respond_and_feedback(user_input, feedback):
    bot_responses = generate_responses(user_input)
    bot_response = "\n".join(bot_responses)
    log_interaction(user_input, bot_response)

    if feedback:
        provide_feedback(user_input, feedback)

        # If feedback indicates the answer was incorrect or missing
        if feedback.lower() in ["incorrect", "missing", "not helpful"]:
            new_answer = input("Please provide the correct answer: ")
            update_knowledge_base(user_input, new_answer)

    return bot_response

# Function to retrain the model based on stored interactions and feedback
def retrain_model():
    # Load interactions and feedback for retraining
    interactions = []
    feedback_storage = {}

    if os.path.exists(interaction_log_path):
        with open(interaction_log_path, 'r') as log_file:
            interactions = log_file.readlines()

    if os.path.exists(feedback_storage_path):
        with open(feedback_storage_path, 'r') as file:
            feedback_storage = json.load(file)

    # Here, we could analyze interactions and feedback to identify gaps in knowledge
    print("Retraining complete with the updated knowledge base.")

# Function to periodically retrain the model
def scheduled_retraining(interval=3600):
    while True:
        time.sleep(interval)  # Wait for the specified interval
        retrain_model()  # Call the retrain function

# Start the retraining in a separate thread
retraining_thread = threading.Thread(target=scheduled_retraining, args=(3600,), daemon=True)
retraining_thread.start()

# Chatbot interface function
def chatbot_interface(user_input, feedback):
    return respond_and_feedback(user_input, feedback)

# Setting up Gradio interface
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=[
        gr.Textbox(label="User Input", placeholder="Type your question here..."),
        gr.Textbox(label="Feedback (optional)", placeholder="Type your feedback here...")
    ],
    outputs="text",
    title="Amity University Chatbot",
    description="Ask questions about Amity University and provide feedback to improve responses."
)

# Launch the interface
iface.launch()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://556e806e2c7e946da8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
